In [20]:
%matplotlib qt

import os
import datetime
from pathlib import Path
import matplotlib.pyplot as mpl
import numpy as np
import toml
from pylibrary.plotting import plothelpers as PH
print(os.getcwd())
os.chdir("/Users/pbmanis/Desktop/Python/VCNModel")
config = toml.load(open("wheres_my_data.toml", "r"))
print(config)
os.chdir(config['codeDirectory'])
import vcnmodel.plotters.plot_sims as plot_sims
import vcnmodel.plotters.figure_data as FD
from vcnmodel.analyzers import sac as SAC
PS = plot_sims.PlotSims(parent=None)
sacs = FD.figure_SAC
print(sacs)
import importlib
importlib.reload(plot_sims)
importlib.reload(SAC)
import pandas as pd

/Users/pbmanis/Desktop/Python/VCNModel
{'baseDataDirectory': '/Volumes/Pegasus_002/VCN-SBEM-Data', 'cellDataDirectory': '/Volumes/Pegasus_002/VCN-SBEM-Data/VCN_Cells', 'revcorrDataDirectory': '/Volumes/Pegasus_002/VCN-SBEM-Data//VCN_Revcorr_Summaries', 'baseMorphologyDirectory': '/Volumes/Pegasus_002/VCN-SBEM-Data/MorphologyData/VCN', 'codeDirectory': '/Users/pbmanis/Desktop/Python/vcnmodel', 'pngDirectory': 'VCN-CellImages-1-2020/Bushy_Cells', 'impedanceDirectory': 'Impedance_Calculations', 'dendriteQualityFile': 'Dendrite Quality and Surface Areas_comparisons_17_June_2021.xlsx', 'asaData': 'Input_ASA_FromautoASA.py', 'asaHeaderSkip': 0, 'SomaAndDendriteData': 'SomaDendrite_Summary'}
{2: ['runANPSTH-all-2022-01-02.19-56-54', 'runANPSTH-largestonly-2022-01-02.20-02-29', 'runANPSTH-removelargest-2022-01-02.20-08-14'], 5: ['runANPSTH-all-2022-01-02.20-14-23', 'runANPSTH-largestonly-2022-01-02.20-21-12', 'runANPSTH-removelargest-2022-01-02.20-27-52'], 6: ['runANPSTH-all-2022-01-02.20-34-5

In [100]:

def one_sac(j, protocol, pname): 

    plot_win = [0.1, 1.0]
    plot_dur = np.fabs(np.diff(plot_win))
    time_scale = 1.0
    gbc = f"VCN_c{int(j):02d}"
    simfile = None
    for k in range(len(sacs[j])):
        if pname in sacs[j][k]:
            simfile = sacs[j][k]
            break
    if simfile is None:
        return None
    basefn = f"{config['cellDataDirectory']:s}/{gbc:s}/Simulations/AN/"
    fndir = Path(basefn, simfile)
    files = list(fndir.glob("*.p"))
    fn = Path(fndir, files[0])
    PD = plot_sims.PData()
    print(f"Getting data for gbc: {gbc:s}")
    SC, syninfo = PS.get_synaptic_info(gbc)
    mtime = Path(fn).stat().st_mtime
    timestamp_str = datetime.datetime.fromtimestamp(mtime).strftime(
        "%Y-%m-%d-%H:%M"
    )

    changetimestamp = plot_sims.get_changetimestamp()

    res = PS.get_data(fn, PD, changetimestamp, protocol)
    if res is None:
        print("no file?")
        return
    (d, AR, SP, RMA, RCP, RCD) = res
    PD.thiscell = gbc
    si = d["Params"]
    ri = d["runInfo"]
    RCP.si = si
    RCP.ri = ri
    (
        totaldur,
        soundtype,
        pip_start,
        pip_duration,
        F0,
        dB,
        fmod,
        dmod,
    ) = PS.get_stim_info(si, ri)
    all_bu_st = []
    all_bu_st_trials = []
    ntr = len(AR.MC.traces)  # number of trials
    for i in range(ntr):  # for all trials in the measure.
        time_base = AR.MC.time_base / 1000.0  # convert to seconds
        trd = d["Results"][i]
        dt = si.dtIC / 1000.0  # convert from msec to seconds
        idx = (int(plot_win[0] / dt), int(plot_win[1] / dt))
        if i == 0:
            waveform = trd["stimWaveform"]
        stb = trd["stimTimebase"]  # convert to seconds
        stim_dt = stb[1] - stb[0]
        if i == 0:
            n_inputs = len(trd["inputSpikeTimes"])
        if len(trd["spikeTimes"]) > 0 and np.nanmax(trd["spikeTimes"]) > 2.0:  # probably in msec
            time_scale = 1e-3  # so scale to seconds
        sptimes = np.array(trd["spikeTimes"]) * time_scale  # convert to seconds
        if not isinstance(trd["spikeTimes"], list) and not isinstance(
            trd["spikeTimes"], np.ndarray
        ):
            cprint("r", "spiketimes is not list")
            cprint("r", f"    {type(trd['spikeTimes'])=}")
            return
        all_bu_st.extend(sptimes)
        all_bu_st_trials.append(sptimes)
        ispt = [  # plot spike times in the SAC analysis window
            i
            for i in range(len(sptimes))
            if sptimes[i] >= pip_start and sptimes[i] < pip_duration - pip_start
        ]
        # P.axdict["B"].plot(
        #     np.array(sptimes[ispt]),
        #     i * np.ones(len(ispt)),
        #     "|",
        #     markersize=1.5,
        #     color="b",
        # )
        w_tb = np.linspace(0.0, stim_dt * len(waveform), num=len(waveform))

        i_wpt = np.where((w_tb > pip_start) & (w_tb <= pip_duration))[0]
        # P.axdict["C"].plot(w_tb[i_wpt], waveform[i_wpt], linewidth=0.33)
    if ri.soundtype.endswith("Clicks"):
        print("Clickpars")
        pars = {
            "twin": 0.002,
            "binw": 3 * dt,
            "delay": ri.clickStart + 0.2 * ri.clickTrainDuration,
            "dur": 0.8 * ri.clickTrainDuration,
            "displayDuration": 0.002,
            "nrep": len(all_bu_st_trials),
            "baseper": 1e-3 * 1.3333333333333333,
            "stimdur": pip_duration * 0.8,
            "maxn": 100000000,
        }

    else:
        print("Sam Pars")
        pars = {
            "twin": 0.020,
            "binw": 3 * dt,
            "delay": pip_start + 0.2 * pip_duration,
            "dur": 0.8 * pip_duration,
            "displayDuration": 0.050,
            "nrep": len(all_bu_st_trials),
            "baseper": 1e-3 * 1.3333333333333333,
            "stimdur": pip_duration * 0.8,
            "maxn": 100000000,
        }
    print(ri.soundtype)
    sac = SAC.SAC()
    yh, bins = sac.SAC_with_histo(
        all_bu_st_trials, pars=pars, engine="cython", dither=dt / 2.0
    )
    sac_bu_CI, peaks, HW, FW = sac.SAC_measures(yh, bins)
    if not np.isnan(sac_bu_CI):
        sac_bu_hw = HW[0][0]*pars["binw"]
        print("BU SAC Report: \n  ")
        print(f"    HW:    {sac_bu_hw:.6f}  CI: {sac_bu_CI:.2f}  Left IPS: {HW[2][0]:.2f}  Right IPS: {HW[3][0]:.2f}")
        # Fs: float = 100e3  # cochlea/zilany model rate
        # F0: float = 16000.0  # stimulus frequency
        # dB: float = 30.0  # in SPL
        # RF: float = 2.5e-3  # rise-fall time
        # fmod: float = 20  # hz, modulation if SAM
        # dmod: float = 0.0  # percent if SAM
        if ri.soundtype.endswith("Clicks"):
            sac_label = f"Expt: {ri.Spirou:14s} {ri.dB:3.0f} dBSPL  HW={1e3*sac_bu_hw:.3f} ms  CI={sac_bu_CI:6.2f}"
        else:
            sac_label = f"Expt: {ri.Spirou:14s} {ri.dB:3.0f} dBSPL Fmod={ri.fmod:5.1}fHz Dmod={ri.dmod:5.1f}\%"
    else:
        print("No spikes ")
        sac_bu_hw = np.nan
    res = {"CI": sac_bu_CI, "HW": sac_bu_hw, "yh": yh, "bins": bins, "simfile": simfile, "gbc": gbc}
    return res





In [98]:

colors = ["blue", "orange", "green"]
expts = ["all", "removelargest", "largestonly"]

def do_plot(df, sacs):
    rowassign = {2:[0,0], 5:[1,0], 6:[2,0], 9:[3,0], 10:[4,0], 11:[0,1], 13:[1,1], 17:[2,1], 18:[3,1], 30:[4,1]}

    P = PH.regular_grid(
        5,
        2,
        order="rowsfirst",
        figsize=(6.0, 8.0),
        showgrid=False,
        verticalspacing=0.05,
        horizontalspacing=0.1,
        margins={
            "bottommargin": 0.1,
            "leftmargin": 0.07,
            "rightmargin": 0.05,
            "topmargin": 0.15,
        },
        labelposition=(0.0, 0.0),
        parent_figure=None,
        panel_labels=None,
    )
    for j in sacs:
        igbc = int(j)
        this_ax = P.axarr[rowassign[igbc][0], rowassign[igbc][1]]
        for ik, k in enumerate(["all",  "removelargest", "largestonly"]):
            if igbc == 30:
                label = k
            else:
                label = None
            bins = df[(df["Cell"] == j) & (df["Expt"] == k)]["bins"]
            yh   = df[(df["Cell"] == j) & (df["Expt"] == k)]["SAC"]
            this_ax.plot(
                bins.values[0]*1e3,
                yh.values[0],
                color=colors[ik],
                label = label
            )

            if ik == 0:
                this_ax.set_title(f"Cell {j:d}")
            if rowassign[igbc][0] == 4:
                this_ax.set_xlabel("msec")
                this_ax.set_ylabel("CI")
    linax = P.axarr.flatten()
    for ax in linax:
        ax.set_ylim(0, max_CI)
    ax.legend(fontsize=7)
    mpl.show()



def run_sac_analysis():            
    protocol = 'runANPSTH'
    rowcount = 0


    max_CI = 0.
    df = pd.DataFrame(data=None, columns=["Cell", "Expt", "CI", "Halfwidth", "SAC", "bins"])

    for j in sacs:
        for ik, k in enumerate(["all", "removelargest", "largestonly"]):
            res = one_sac(j, protocol, k)
            igbc = int(res["gbc"][-2:])
            if res["CI"] == 0:
                res["CI"] = np.nan
            if res is not None:
                if res["CI"] == 0:
                    res["CI"] = np.nan

                print("gbc: ", res["gbc"], " simfile: ", res["simfile"])
                # P.axarr[rowassign[igbc][0], rowassign[igbc][1]].plot(res["bins"], res["yh"], color=colors[ik])
                if res["CI"] > max_CI:
                    max_CI = res["CI"]
                df = df.append({"Cell": j, "Expt": k, "CI": res["CI"], "Halfwidth": res["HW"], "SAC": res["yh"], 
                    'bins': res["bins"]}, ignore_index=True)
            else:
                df = df.append({"Cell": j, "Expt": k, "CI": np.nan, "Halfwidth": np.nan , "SAC": res["yh"], 
                    'bins': res["bins"]}, ignore_index=True)

    return df

# df

In [99]:
do_plot(df, sacs)
print(sacs)

{2: ['runANPSTH-all-2022-01-02.19-56-54', 'runANPSTH-largestonly-2022-01-02.20-02-29', 'runANPSTH-removelargest-2022-01-02.20-08-14'], 5: ['runANPSTH-all-2022-01-02.20-14-23', 'runANPSTH-largestonly-2022-01-02.20-21-12', 'runANPSTH-removelargest-2022-01-02.20-27-52'], 6: ['runANPSTH-all-2022-01-02.20-34-56', 'runANPSTH-largestonly-2022-01-02.20-39-59', 'runANPSTH-removelargest-2022-01-02.20-45-00'], 9: ['runANPSTH-all-2022-01-02.20-50-30', 'runANPSTH-largestonly-2022-01-02.20-56-36', 'runANPSTH-removelargest-2022-01-02.21-02-45'], 10: ['runANPSTH-all-2022-01-02.21-09-25', 'runANPSTH-largestonly-2022-01-02.21-16-25', 'runANPSTH-removelargest-2022-01-02.21-23-22'], 11: ['runANPSTH-all-2022-01-02.21-30-35', 'runANPSTH-largestonly-2022-01-02.21-35-34', 'runANPSTH-removelargest-2022-01-02.21-40-28'], 13: ['runANPSTH-all-2022-01-02.21-45-49', 'runANPSTH-largestonly-2022-01-02.21-51-00', 'runANPSTH-removelargest-2022-01-02.21-56-14'], 17: ['runANPSTH-all-2022-01-02.22-02-02', 'runANPSTH-large

In [23]:
import seaborn as sns


,Cell,Expt,CI,Halfwidth
0,2,all,43.986721,0.000379
1,2,largestonly,38.580247,0.000100
2,2,removelargest,39.075409,0.000478
3,5,all,54.395502,0.000381
4,5,largestonly,NaN,NaN
5,5,removelargest,48.430895,0.000435
6,6,all,34.289136,0.000363
7,6,largestonly,15.950546,0.000000
8,6,removelargest,38.835333,0.000394
9,9,all,25.910478,0.000339


In [97]:
import seaborn as sns
from matplotlib import collections  as mc
print(dir(sns))
#sns.barplot(data=df, x="Expt", y="CI", hue="Cell")
df["HWms"] = df["Halfwidth"].values*1000.0
df['HWms'] = df['HWms'].replace(0, np.nan)
PB = PH.regular_grid(
    1,
    2,
    order="rowsfirst",
    figsize=(6.0,4.0),
    showgrid=False,
    verticalspacing=0.05,
    horizontalspacing=0.1,
    margins={
        "bottommargin": 0.1,
        "leftmargin": 0.07,
        "rightmargin": 0.05,
        "topmargin": 0.15,
    },
    labelposition=(0.0, 0.0),
    parent_figure=None,
    panel_labels=None,
)

hue_order = ["all", "removelargest", "largestonly"]

sns.stripplot(data=df, x="Cell", y="HWms", hue="Expt", ax=PB.axdict["A"], size=7)
# sns.lineplot(data=df, x="Cell", y="HWms", hue="Expt", hue_order=hue_order, ax=PB.axdict["A"])


# lines = ([[x, n] for n in group] for x, (_, group) in enumerate(df.groupby(['Cell'], sort = False)['HWms']))
# lc = mc.LineCollection(lines, colors='red', linewidths=1)    
# PB.axdict["A"].add_collection(lc)

sns.stripplot(data=df, x="Cell", y="CI", hue="Expt", ax=PB.axdict["B"], size=7)





['FacetGrid', 'JointGrid', 'PairGrid', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_core', '_decorators', '_docstrings', '_orig_rc_params', '_statistics', 'algorithms', 'axes_style', 'axisgrid', 'barplot', 'blend_palette', 'boxenplot', 'boxplot', 'categorical', 'catplot', 'choose_colorbrewer_palette', 'choose_cubehelix_palette', 'choose_dark_palette', 'choose_diverging_palette', 'choose_light_palette', 'clustermap', 'cm', 'color_palette', 'colors', 'countplot', 'crayon_palette', 'crayons', 'cubehelix_palette', 'dark_palette', 'desaturate', 'despine', 'displot', 'distplot', 'distributions', 'diverging_palette', 'dogplot', 'ecdfplot', 'external', 'factorplot', 'get_data_home', 'get_dataset_names', 'heatmap', 'histplot', 'hls_palette', 'husl_palette', 'jointplot', 'kdeplot', 'light_palette', 'lineplot', 'lmplot', 'load_dataset', 'matrix', 'miscplot', 'move_legend', 'mpl', 'mpl_palette', 'pairplot', '

<AxesSubplot:xlabel='Cell', ylabel='CI'>